In [1]:
from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [3]:
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)



    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    

    return Model(noise, img)

In [4]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [5]:
def train(epochs, batch_size=128, save_interval=500):
    (x_train,_),(_,_)=mnist.load_data()
    
    x_train=(x_train.astype(np.float32)-127.5)/ 127.5
    
    X_train= np.expand_dims(x_train, axis=3)
    
    valid=np.ones((batch_size, 1))
    fake= np.zeros((batch_size, 1))
    
    
    for epoch in range(epochs):
        idx= np.random.randint(0, X_train.shape[0], batch_size)
        imgs= X_train[idx]
        
        ###Train Discriminator
        noise= np.random.normal(0, 1, (batch_size, 100))
        gen_imgs= generator.predict(noise)
        
        d_loss_real= discriminator.train_on_batch(imgs, valid)
        d_loss_fake= discriminator.train_on_batch(gen_imgs, fake)
        d_loss= 0.5 * np.add(d_loss_real, d_loss_fake)
        
        
        ####Train Generator
        noise= np.random.normal(0, 1, (batch_size, 100))
        g_loss= combined.train_on_batch(noise, valid)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % save_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images_mnist/mnist_%d.png" % epoch)
    plt.close()


In [6]:
optimizer = Adam(0.0002, 0.5)

In [7]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

In [8]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [9]:
z = Input(shape=(100,))   
img = generator(z)

In [10]:
discriminator.trainable=False

In [11]:
valid= discriminator(img)

In [12]:
combined=Model(z, valid)
combined.compile(loss="binary_crossentropy", optimizer=optimizer)

In [13]:
train(epochs=1000, batch_size=128, save_interval=1000)

4/4 [==============================] - 1s 3ms/step
0 [D loss: 0.674958, acc.: 39.84%] [G loss: 0.645996]
4/4 [==============================] - 0s 2ms/step
1 [D loss: 0.369189, acc.: 78.52%] [G loss: 0.651272]
4/4 [==============================] - 0s 3ms/step
2 [D loss: 0.347697, acc.: 80.47%] [G loss: 0.669643]
4/4 [==============================] - 0s 3ms/step
3 [D loss: 0.345157, acc.: 78.52%] [G loss: 0.697292]
4/4 [==============================] - 0s 2ms/step
4 [D loss: 0.325464, acc.: 85.55%] [G loss: 0.807510]
4/4 [==============================] - 0s 2ms/step
5 [D loss: 0.306574, acc.: 87.89%] [G loss: 0.902706]
4/4 [==============================] - 0s 2ms/step
6 [D loss: 0.267947, acc.: 97.27%] [G loss: 1.039303]
4/4 [==============================] - 0s 2ms/step
7 [D loss: 0.230194, acc.: 99.61%] [G loss: 1.167401]
4/4 [==============================] - 0s 2ms/step
8 [D loss: 0.205667, acc.: 98.44%] [G loss: 1.314104]
4/4 [==============================] - 0s 2ms/step
9 [D

4/4 [==============================] - 0s 2ms/step
76 [D loss: 0.043482, acc.: 99.61%] [G loss: 4.501525]
4/4 [==============================] - 0s 2ms/step
77 [D loss: 0.075792, acc.: 98.05%] [G loss: 4.102776]
4/4 [==============================] - 0s 1ms/step
78 [D loss: 0.043521, acc.: 98.44%] [G loss: 4.658410]
4/4 [==============================] - 0s 1ms/step
79 [D loss: 0.178254, acc.: 92.58%] [G loss: 3.639587]
4/4 [==============================] - 0s 2ms/step
80 [D loss: 0.080565, acc.: 95.70%] [G loss: 4.257212]
4/4 [==============================] - 0s 2ms/step
81 [D loss: 0.023335, acc.: 100.00%] [G loss: 4.444098]
4/4 [==============================] - 0s 1ms/step
82 [D loss: 0.059795, acc.: 98.05%] [G loss: 4.218328]
4/4 [==============================] - 0s 2ms/step
83 [D loss: 0.035459, acc.: 99.22%] [G loss: 4.125216]
4/4 [==============================] - 0s 2ms/step
84 [D loss: 0.096468, acc.: 95.31%] [G loss: 4.273153]
4/4 [==============================] - 0s 1ms

4/4 [==============================] - 0s 1ms/step
153 [D loss: 0.207846, acc.: 91.41%] [G loss: 2.969220]
4/4 [==============================] - 0s 1ms/step
154 [D loss: 0.499811, acc.: 77.34%] [G loss: 2.262809]
4/4 [==============================] - 0s 1ms/step
155 [D loss: 0.199023, acc.: 91.80%] [G loss: 2.868109]
4/4 [==============================] - 0s 1ms/step
156 [D loss: 0.740137, acc.: 62.50%] [G loss: 1.586118]
4/4 [==============================] - 0s 1ms/step
157 [D loss: 0.296703, acc.: 82.81%] [G loss: 2.623608]
4/4 [==============================] - 0s 1ms/step
158 [D loss: 0.222768, acc.: 93.36%] [G loss: 2.852971]
4/4 [==============================] - 0s 1ms/step
159 [D loss: 0.485516, acc.: 73.44%] [G loss: 2.473812]
4/4 [==============================] - 0s 1ms/step
160 [D loss: 0.339044, acc.: 85.16%] [G loss: 2.666996]
4/4 [==============================] - 0s 1ms/step
161 [D loss: 0.508657, acc.: 73.83%] [G loss: 1.876994]
4/4 [==============================] 

4/4 [==============================] - 0s 1ms/step
230 [D loss: 0.696102, acc.: 45.70%] [G loss: 0.725000]
4/4 [==============================] - 0s 1ms/step
231 [D loss: 0.694816, acc.: 46.09%] [G loss: 0.712239]
4/4 [==============================] - 0s 2ms/step
232 [D loss: 0.708014, acc.: 44.14%] [G loss: 0.688816]
4/4 [==============================] - 0s 2ms/step
233 [D loss: 0.704559, acc.: 43.75%] [G loss: 0.678782]
4/4 [==============================] - 0s 2ms/step
234 [D loss: 0.732413, acc.: 43.75%] [G loss: 0.654291]
4/4 [==============================] - 0s 2ms/step
235 [D loss: 0.718196, acc.: 46.09%] [G loss: 0.637878]
4/4 [==============================] - 0s 2ms/step
236 [D loss: 0.725504, acc.: 43.36%] [G loss: 0.614769]
4/4 [==============================] - 0s 2ms/step
237 [D loss: 0.691876, acc.: 47.66%] [G loss: 0.641787]
4/4 [==============================] - 0s 2ms/step
238 [D loss: 0.712606, acc.: 44.14%] [G loss: 0.622032]
4/4 [==============================] 

4/4 [==============================] - 0s 0s/step
307 [D loss: 0.658557, acc.: 48.44%] [G loss: 0.651088]
4/4 [==============================] - 0s 3ms/step
308 [D loss: 0.666917, acc.: 48.83%] [G loss: 0.655318]
4/4 [==============================] - 0s 0s/step
309 [D loss: 0.658723, acc.: 49.22%] [G loss: 0.650814]
4/4 [==============================] - 0s 3ms/step
310 [D loss: 0.659841, acc.: 48.05%] [G loss: 0.650794]
4/4 [==============================] - 0s 0s/step
311 [D loss: 0.664877, acc.: 48.44%] [G loss: 0.645193]
4/4 [==============================] - 0s 2ms/step
312 [D loss: 0.654781, acc.: 49.61%] [G loss: 0.636680]
4/4 [==============================] - 0s 0s/step
313 [D loss: 0.670283, acc.: 48.83%] [G loss: 0.636008]
4/4 [==============================] - 0s 0s/step
314 [D loss: 0.659882, acc.: 49.61%] [G loss: 0.637829]
4/4 [==============================] - 0s 0s/step
315 [D loss: 0.668456, acc.: 49.22%] [G loss: 0.637493]
4/4 [==============================] - 0s 3

...

4/4 [==============================] - 0s 339us/step
692 [D loss: 0.597932, acc.: 75.39%] [G loss: 0.808639]
4/4 [==============================] - 0s 0s/step
693 [D loss: 0.640222, acc.: 62.50%] [G loss: 0.809355]
4/4 [==============================] - 0s 2ms/step
694 [D loss: 0.630668, acc.: 67.58%] [G loss: 0.810159]
4/4 [==============================] - 0s 0s/step
695 [D loss: 0.611155, acc.: 71.09%] [G loss: 0.818148]
4/4 [==============================] - 0s 0s/step
696 [D loss: 0.633504, acc.: 66.80%] [G loss: 0.829276]
4/4 [==============================] - 0s 0s/step
697 [D loss: 0.627814, acc.: 69.14%] [G loss: 0.823839]
4/4 [==============================] - 0s 0s/step
698 [D loss: 0.617804, acc.: 67.19%] [G loss: 0.832826]
4/4 [==============================] - 0s 667us/step
699 [D loss: 0.637915, acc.: 62.11%] [G loss: 0.842812]
4/4 [==============================] - 0s 5ms/step
700 [D loss: 0.623654, acc.: 68.75%] [G loss: 0.838262]
4/4 [==============================] -

4/4 [==============================] - 0s 1ms/step
769 [D loss: 0.625874, acc.: 68.36%] [G loss: 0.837891]
4/4 [==============================] - 0s 1ms/step
770 [D loss: 0.589676, acc.: 75.78%] [G loss: 0.853461]
4/4 [==============================] - 0s 1ms/step
771 [D loss: 0.606901, acc.: 72.27%] [G loss: 0.857190]
4/4 [==============================] - 0s 3ms/step
772 [D loss: 0.598549, acc.: 75.39%] [G loss: 0.855373]
4/4 [==============================] - 0s 1ms/step
773 [D loss: 0.594618, acc.: 71.88%] [G loss: 0.848981]
4/4 [==============================] - 0s 1ms/step
774 [D loss: 0.592310, acc.: 73.05%] [G loss: 0.860139]
4/4 [==============================] - 0s 1ms/step
775 [D loss: 0.585393, acc.: 75.39%] [G loss: 0.873754]
4/4 [==============================] - 0s 3ms/step
776 [D loss: 0.598930, acc.: 74.61%] [G loss: 0.867679]
4/4 [==============================] - 0s 1ms/step
777 [D loss: 0.591924, acc.: 73.44%] [G loss: 0.883066]
4/4 [==============================] 

4/4 [==============================] - 0s 2ms/step
846 [D loss: 0.601151, acc.: 69.53%] [G loss: 0.909891]
4/4 [==============================] - 0s 2ms/step
847 [D loss: 0.613198, acc.: 68.75%] [G loss: 0.919652]
4/4 [==============================] - 0s 3ms/step
848 [D loss: 0.602851, acc.: 74.22%] [G loss: 0.939234]
4/4 [==============================] - 0s 2ms/step
849 [D loss: 0.613419, acc.: 71.09%] [G loss: 0.932904]
4/4 [==============================] - 0s 2ms/step
850 [D loss: 0.590347, acc.: 76.56%] [G loss: 0.942961]
4/4 [==============================] - 0s 2ms/step
851 [D loss: 0.599618, acc.: 74.22%] [G loss: 0.921732]
4/4 [==============================] - 0s 1ms/step
852 [D loss: 0.591954, acc.: 81.25%] [G loss: 0.919775]
4/4 [==============================] - 0s 2ms/step
853 [D loss: 0.593218, acc.: 74.61%] [G loss: 0.915030]
4/4 [==============================] - 0s 2ms/step
854 [D loss: 0.591618, acc.: 74.61%] [G loss: 0.896895]
4/4 [==============================] 

922 [D loss: 0.606828, acc.: 69.14%] [G loss: 0.919715]
4/4 [==============================] - 0s 1ms/step
923 [D loss: 0.586899, acc.: 72.66%] [G loss: 0.936693]
4/4 [==============================] - 0s 1ms/step
924 [D loss: 0.583354, acc.: 71.48%] [G loss: 0.965253]
4/4 [==============================] - 0s 2ms/step
925 [D loss: 0.631312, acc.: 62.89%] [G loss: 0.917761]
4/4 [==============================] - 0s 2ms/step
926 [D loss: 0.616225, acc.: 67.97%] [G loss: 0.904392]
4/4 [==============================] - 0s 2ms/step
927 [D loss: 0.584788, acc.: 72.66%] [G loss: 0.902397]
4/4 [==============================] - 0s 2ms/step
928 [D loss: 0.587997, acc.: 71.48%] [G loss: 0.899284]
4/4 [==============================] - 0s 2ms/step
929 [D loss: 0.578231, acc.: 71.88%] [G loss: 0.911849]
4/4 [==============================] - 0s 2ms/step
930 [D loss: 0.605362, acc.: 66.41%] [G loss: 0.899537]
4/4 [==============================] - 0s 1ms/step
931 [D loss: 0.563291, acc.: 77.34%] 

4/4 [==============================] - 0s 1ms/step
999 [D loss: 0.613039, acc.: 63.67%] [G loss: 0.965441]
